In [40]:
# Libraries
from nltk.util import pr
import pandas as pd #Untuk mengimport data/file (csv, dll) dari sumber luar
import numpy as np #mengelola komputasi matrix
import re #reguler expression
import string
import nltk #Natural language Tool Kit untuk proses tekonisasi, filtering
import demoji
import emoji
from emoji import demojize
import pickle #untuk menyimpan hasil model
from nltk.corpus import stopwords
from nltk import word_tokenize

In [43]:
data = pd.read_csv('C:/Users/sinag/SKRIPSIKU/data_train.csv', sep = ',', encoding ='utf-8')
data.head()

,id,text,sentimen,label
0,1,Rame2 dislike berita corona,negatif,-1
1,2,Pilih mati kena korona atau mati kelaparan se...,negatif,-1
2,3,Pembahasan penanganan ATAU membuat projek bisn...,negatif,-1
3,4,mari membantu pemerintah agar virus corona seg...,positif,1
4,5,Gak ada satu pun hal positif yg disampaikan pe...,negatif,-1


In [44]:
data.shape

(1069, 4)

In [31]:
nan_rows = data[data['label'].isnull()]
print(nan_rows)

Empty DataFrame
Columns: [id, username, text, sentimen, label]
Index: []


In [22]:
data['sentimen'].value_counts()

negatif    3096
positif    1208
netral     1044
Name: sentimen, dtype: int64

In [23]:
# proses case folding 
def casefolding(text):
    text = text.lower()
    return text
data['text'] = data['text'].apply(casefolding)
data.head()

,id,username,text,sentimen,label
0,1,akhmad Zubaeri,repot .!untuk orang miskin kayak saya yang ker...,negatif,-1
1,2,Amin Udin,stiap desa kasih relawan dan alat tes .pak aga...,positif,1
2,3,Amin Udin,kesadaran masyarakat yang rendah tapi yg di sa...,negatif,-1
3,4,Amin Udin,semangat terus pak jokowi,netral,0
4,5,Amin Udin,pengen hidup normal lagi,netral,0


In [24]:
# proses cleansing remove regex (cleansing) seperti tanda baca dan angka angka
# Proses Cleansing (menghapus karaketer selain huruf)
def cleansing(text):
    text = re.sub(r'@[\w|a-z0-9]*', ' ', text) #menghapus @username      
    text = re.sub(r'#[\w|a-z0-9]*', ' ', text) #menghapus #hashtags    
    text = re.sub(r'https://*[\r\n]*', ' ', text) #menghapus link http 
    text = re.sub(r'www.[^ ]+', ' ', text) #menhilangkan situs website 
    text = re.sub(r"\d+", "", text) # menghapus bilangan
    text = text.translate(str.maketrans("","",string.punctuation)) #menghapus simbol  
    text = re.sub(r'\r', ' ', text) #menghapus \r    
    text = re.sub(r'\n', ' ', text) #menghapus \n (newline)   
    text = demoji.replace(text, ' ') #menghapus emoji
    return text
data['text'] = data['text'].apply(cleansing)
data.head()

,id,username,text,sentimen,label
0,1,akhmad Zubaeri,repot untuk orang miskin kayak saya yang kerja...,negatif,-1
1,2,Amin Udin,stiap desa kasih relawan dan alat tes pak agar...,positif,1
2,3,Amin Udin,kesadaran masyarakat yang rendah tapi yg di sa...,negatif,-1
3,4,Amin Udin,semangat terus pak jokowi,netral,0
4,5,Amin Udin,pengen hidup normal lagi,netral,0


In [25]:
def open_kamus_prepro(x):
	kamus={}
	with open(x,'r') as file :
		for line in file :
			slang=line.replace("'","").split(':')
			kamus[slang[0].strip()]=slang[1].rstrip('\n').lstrip()
	return kamus

kamus_slang=open_kamus_prepro('C:/Users/sinag/SKRIPSIKU/slangwords.txt')

In [26]:
def slangword(text):  
    sentence_list = text.split()
    new_sentence = []
    for word in sentence_list:
      for candidate_replacement in kamus_slang:
        if candidate_replacement == word:
          word = word.replace(candidate_replacement, kamus_slang[candidate_replacement])
      new_sentence.append(word)
    return " ".join(new_sentence)

data['text'] = data['text'].apply(lambda x: slangword(x))
data.head()

,id,username,text,sentimen,label
0,1,akhmad Zubaeri,repot untuk orang miskin kayak saya yang kerja...,negatif,-1
1,2,Amin Udin,setiap desa kasih relawan dan alat tes pak aga...,positif,1
2,3,Amin Udin,kesadaran masyarakat yang rendah tapi yg di sa...,negatif,-1
3,4,Amin Udin,semangat terus pak jokowi,netral,0
4,5,Amin Udin,ingin hidup normal lagi,netral,0


In [27]:
# Proses Tokenizing (memecah kalimat menjadi kata)
def tokenizing(text):
    text = nltk.tokenize.word_tokenize (text)
    return text
data['text'] = data ['text'].apply(tokenizing)
data.head()

,id,username,text,sentimen,label
0,1,akhmad Zubaeri,"[repot, untuk, orang, miskin, kayak, saya, yan...",negatif,-1
1,2,Amin Udin,"[setiap, desa, kasih, relawan, dan, alat, tes,...",positif,1
2,3,Amin Udin,"[kesadaran, masyarakat, yang, rendah, tapi, yg...",negatif,-1
3,4,Amin Udin,"[semangat, terus, pak, jokowi]",netral,0
4,5,Amin Udin,"[ingin, hidup, normal, lagi]",netral,0


In [28]:
# Menambah daftar stopword 
stopword = open('C:/Users/sinag/SKRIPSIKU/stopwords.txt',"r",encoding="utf-8", errors='replace')
stopword = [t.strip('\n').strip() for t in stopword]

# Proses Filtering/Stopword (membuang kata yang tidak dibutuhkan)
def stopword_removal(text):
    filtering = stopword
    x = []
    data = []
    def myFunc(x):
        if x in filtering:
            return False
        else:
            return True
    fit = filter(myFunc, text)
    for x in fit:
        data.append(x)
    return data
data['text'] = data ['text'].apply(stopword_removal)
data.head()

,id,username,text,sentimen,label
0,1,akhmad Zubaeri,"[repot, orang, miskin, kerjanya, serabutan]",negatif,-1
1,2,Amin Udin,"[setiap, desa, kasih, relawan, alat, tes, pak,...",positif,1
2,3,Amin Udin,"[kesadaran, masyarakat, rendah, salahkan, mala...",negatif,-1
3,4,Amin Udin,"[semangat, pak, jokowi]",netral,0
4,5,Amin Udin,"[ingin, hidup, normal]",netral,0


In [29]:
data.to_csv('C:/Users/sinag/SKRIPSIKU/preprocess.csv',index=False)

In [30]:
data_clean = pd.read_csv('C:/Users/sinag/SKRIPSIKU/preprocess.csv', encoding='latin1')
data_clean.head()

,id,username,text,sentimen,label
0,1,akhmad Zubaeri,"['repot', 'orang', 'miskin', 'kerjanya', 'sera...",negatif,-1
1,2,Amin Udin,"['setiap', 'desa', 'kasih', 'relawan', 'alat',...",positif,1
2,3,Amin Udin,"['kesadaran', 'masyarakat', 'rendah', 'salahka...",negatif,-1
3,4,Amin Udin,"['semangat', 'pak', 'jokowi']",netral,0
4,5,Amin Udin,"['ingin', 'hidup', 'normal']",netral,0


In [31]:
# Proses Cleansing terakhir (untuk menghapus koma)
def clean(text):
    text = text.translate(str.maketrans("","",string.punctuation)) #menghapus simbol
    return text
data_clean['text'] = data_clean['text'].apply(clean)
data_clean.head()

,id,username,text,sentimen,label
0,1,akhmad Zubaeri,repot orang miskin kerjanya serabutan,negatif,-1
1,2,Amin Udin,setiap desa kasih relawan alat tes pak semua t...,positif,1
2,3,Amin Udin,kesadaran masyarakat rendah salahkan malah pem...,negatif,-1
3,4,Amin Udin,semangat pak jokowi,netral,0
4,5,Amin Udin,ingin hidup normal,netral,0


In [32]:
data_clean.to_csv('C:/Users/sinag/SKRIPSIKU/data_bersih.csv',index=False)

In [33]:
data_clean = pd.read_csv('C:/Users/sinag/SKRIPSIKU/data_bersih.csv', encoding='latin1')
data_clean.head()

,id,username,text,sentimen,label
0,1,akhmad Zubaeri,repot orang miskin kerjanya serabutan,negatif,-1
1,2,Amin Udin,setiap desa kasih relawan alat tes pak semua t...,positif,1
2,3,Amin Udin,kesadaran masyarakat rendah salahkan malah pem...,negatif,-1
3,4,Amin Udin,semangat pak jokowi,netral,0
4,5,Amin Udin,ingin hidup normal,netral,0


In [33]:
data_clean.shape

(1337, 5)